In [1]:
import scrapy
import requests
from scrapy.http import TextResponse

In [144]:
# IR GO 홈페이지 연결
url = "https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C"
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C>

In [145]:
name = response.xpath('//*[@id="irDataList"]/div/div[2]/span[1]/text()').extract()
date = response.xpath('//*[@id="irDataList"]/div/div[2]/span[3]/text()').extract()
links = response.xpath('//*[@id="irDataList"]/div/@data-href').extract()
links

['https://m.irgo.co.kr/IR자료/28195/TB/호전실업-호전실업-신영-2019-하반기-Corporate-day-참가국내',
 'https://m.irgo.co.kr/IR자료/28197/TB/BNK금융지주-BNK금융지주-2019년-3분기-경영실적-및-주요-관심사항싱가포르-홍콩',
 'https://m.irgo.co.kr/IR자료/28199/TB/현대모비스-현대모비스-Shinhan-Super-Corporate-Day-참석국내',
 'https://m.irgo.co.kr/IR자료/28194/TB/현대제철-현대제철-2019-APAC-Resources-Conference-참석중국',
 'https://m.irgo.co.kr/IR자료/28201/TB/코스맥스-코스맥스-신영-2019-하반기-CORPORATE-DAY-참가국내',
 'https://m.irgo.co.kr/IR자료/28202/TB/코스맥스비티아이-코스맥스비티아이-신영-2019-하반기-CORPORATE-DAY-참가국내',
 'https://m.irgo.co.kr/IR자료/28200/TB/브레인콘텐츠-브레인콘텐츠-회사현황-및-주요-경영현황-설명국내',
 'https://m.irgo.co.kr/IR자료/28198/TB/디케이앤디-디케이앤디-2019년-3분기-실적-설명국내',
 'https://m.irgo.co.kr/IR자료/28181/TB/신성이엔지-이젠-공기청정기도-빌트인-시대-B2C까지-사업-영역을-확장한-신성이엔지',
 'https://m.irgo.co.kr/IR자료/28196/TB/코리아센터-코리아센터-신한-Corporate-Day-참가국내',
 'https://m.irgo.co.kr/IR자료/28179/TB/IRGO-주담톡톡아이에스동서-IR담당자가-직접-전하는-기업-이야기',
 'https://m.irgo.co.kr/IR자료/28188/TB/롯데지주-롯데지주-국내-자산운용-CIO조찬-간담회국내',
 'https://m.irgo.co.kr/IR자료/28187/TB/유니퀘스트-유니퀘스트-주요

### 세부 정보 크롤링

In [67]:
req = requests.get(links[0])

# response 객체 생성
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C/28195/TB/%ED%98%B8%EC%A0%84%EC%8B%A4%EC%97%85-%ED%98%B8%EC%A0%84%EC%8B%A4%EC%97%85-%EC%8B%A0%EC%98%81-2019-%ED%95%98%EB%B0%98%EA%B8%B0-Corporate-day-%EC%B0%B8%EA%B0%80%EA%B5%AD%EB%82%B4>

In [46]:
name = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[1]/a/span[2]/text()')[0].extract()
date = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[2]/text()')[0].extract()
url = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[3]/a/@href')[0].extract()
name, date, url

('디케이앤디',
 '1시간전',
 'https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/2DWXTCB2VQRNWWQ9GMD252EBX92SXCH72019123112156.pdf')

In [79]:
number = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[1]/a/@href')[0].extract()
code="A"+str(number[29:35])
"http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode={}".format(code)

In [80]:
"http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode={}".format(code)

'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A111110'

### 프로젝트 시작

In [2]:
!rm -rf ir_crawler
!scrapy startproject ir_crawler

New Scrapy project 'ir_crawler', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/notebook/project/ir_crawler

You can start your first spider with:
    cd ir_crawler
    scrapy genspider example example.com


In [146]:
!tree ir_crawler

ir_crawler
├── IRCrawler.csv
├── ir_crawler
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── items.cpython-36.pyc
│   │   ├── pipelines.cpython-36.pyc
│   │   └── settings.cpython-36.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── mongodb.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-36.pyc
│       │   └── spider.cpython-36.pyc
│       └── spider.py
└── scrapy.cfg

4 directories, 16 files


#### items.py 작성

In [49]:
%%writefile ir_crawler/ir_crawler/items.py
import scrapy

class CrawlerItem(scrapy.Item):
    name = scrapy.Field()
    date = scrapy.Field()
    ir_link = scrapy.Field()
    fs_link = scrapy.Field()

Overwriting ir_crawler/ir_crawler/items.py


#### spiders.py 작성

In [79]:
%%writefile ir_crawler/ir_crawler/spiders/spider.py
import datetime
import scrapy
from ir_crawler.items import CrawlerItem

class Spider(scrapy.Spider):
    name = "IRCrawler"
    allow_domain = ["https://m.irgo.co.kr/"]
    start_urls = ["https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C"]
    
    def parse(self, response):
        date = response.xpath('//*[@id="irDataList"]/div/div[2]/span[3]/text()').extract()
        urls = response.xpath('//*[@id="irDataList"]/div/@data-href').extract()
        links = []
        for i in range(len(date)):
            if date[i][-3:] == "1일전":
                links.append(urls[i])
        for link in links:
            yield scrapy.Request(link, callback=self.page_content)
            
    def page_content(self, response):
        item = CrawlerItem()
        item["name"] = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[1]/a/span[2]/text()')[0].extract()
        item["date"] = str(datetime.datetime.now())[:10]
        try:
            item["ir_link"] = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[3]/a/@href')[0].extract()
        except:
            item["ir_link"] = response.url
        number = response.xpath('//*[@id="content"]/div[1]/div/dl/dd[1]/a/@href')[0].extract()
        code= "A" + str(number[29:35])
        item["fs_link"] = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode={}".format(code)
        yield item

Overwriting ir_crawler/ir_crawler/spiders/spider.py


#### run.sh 작성 (csv 파일로 저장)

In [80]:
%%writefile run.sh
cd ir_crawler
scrapy crawl IRCrawler -o IRCrawler.csv

Overwriting run.sh


In [81]:
!chmod +x run.sh

In [82]:
!./run.sh

2019-12-04 03:43:36 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: ir_crawler)
2019-12-04 03:43:36 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Oct 24 2019, 05:41:31) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1051-aws-x86_64-with-debian-buster-sid
2019-12-04 03:43:36 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'ir_crawler', 'FEED_FORMAT': 'csv', 'FEED_URI': 'IRCrawler.csv', 'NEWSPIDER_MODULE': 'ir_crawler.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['ir_crawler.spiders']}
2019-12-04 03:43:36 [scrapy.extensions.telnet] INFO: Telnet Password: 6e9cf7192d0de43b
2019-12-04 03:43:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scr

2019-12-04 03:43:39 [scrapy.core.scraper] DEBUG: Scraped from <200 https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C/28185/TB/%ED%95%9C%EC%84%B8%EC%98%88%EC%8A%A424%ED%99%80%EB%94%A9%EC%8A%A4-%ED%95%9C%EC%84%B8%EC%98%88%EC%8A%A424%ED%99%80%EB%94%A9%EC%8A%A4-%EC%8B%A0%EC%98%81%EC%A6%9D%EA%B6%8C-2019%EB%85%84-%ED%95%98%EB%B0%98%EA%B8%B0-Corporate-Day-%EC%B0%B8%EA%B0%80%EA%B5%AD%EB%82%B4>
{'date': '2019-12-04',
 'fs_link': 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A016450',
 'ir_link': 'https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/RJA2TFQSMV4RM43C7TS5U7TEXBFHSPZG20191239447.pdf',
 'name': '한세예스24홀딩스'}
2019-12-04 03:43:39 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C/28188/TB/%EB%A1%AF%EB%8D%B0%EC%A7%80%EC%A3%BC-%EB%A1%AF%EB%8D%B0%EC%A7%80%EC%A3%BC-%EA%B5%AD%EB%82%B4-%EC%9E%90%EC%82%B0%EC%9A%B4%EC%9A%A9-CIO%EC%A1%B0%EC%B0%AC-%EA%B0%84%EB%8B%B4%ED%9A%8C%EA%B5%AD%EB%82%B4> (referer: https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C)
2019-12-04 03

2019-12-04 03:43:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C/28202/TB/%EC%BD%94%EC%8A%A4%EB%A7%A5%EC%8A%A4%EB%B9%84%ED%8B%B0%EC%95%84%EC%9D%B4-%EC%BD%94%EC%8A%A4%EB%A7%A5%EC%8A%A4%EB%B9%84%ED%8B%B0%EC%95%84%EC%9D%B4-%EC%8B%A0%EC%98%81-2019-%ED%95%98%EB%B0%98%EA%B8%B0-CORPORATE-DAY-%EC%B0%B8%EA%B0%80%EA%B5%AD%EB%82%B4> (referer: https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C)
2019-12-04 03:43:41 [scrapy.core.scraper] DEBUG: Scraped from <200 https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C/28202/TB/%EC%BD%94%EC%8A%A4%EB%A7%A5%EC%8A%A4%EB%B9%84%ED%8B%B0%EC%95%84%EC%9D%B4-%EC%BD%94%EC%8A%A4%EB%A7%A5%EC%8A%A4%EB%B9%84%ED%8B%B0%EC%95%84%EC%9D%B4-%EC%8B%A0%EC%98%81-2019-%ED%95%98%EB%B0%98%EA%B8%B0-CORPORATE-DAY-%EC%B0%B8%EA%B0%80%EA%B5%AD%EB%82%B4>
{'date': '2019-12-04',
 'fs_link': 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A044820',
 'ir_link': 'https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/9MDWH49N5X8M5579QUV2C9WBNZZDSTDY2019123143455.p

In [142]:
import pandas as pd
df = pd.read_csv("ir_crawler/IRCrawler.csv")
df

,date,fs_link,ir_link,name
0,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,코스맥스
1,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,유니퀘스트
2,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://m.irgo.co.kr/IR%EC%9E%90%EB%A3%8C/2818...,신성이엔지
3,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,롯데지주
4,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,한세예스24홀딩스
5,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,브레인콘텐츠
6,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,디케이앤디
7,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,코스맥스
8,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,코스맥스비티아이
9,2019-12-04,http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?...,https://file.irgo.co.kr/data/BOARD/ATTACH_PDF/...,현대모비스
